In [ ]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

from typing import Optional, Dict, Any
import os
import time
from pathlib import Path
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder

load_dotenv()

import asyncio

In [ ]:
 # Initialize Azure credentials
credential = DefaultAzureCredential()

In [ ]:
 # Initialize Azure credentials
credential = DefaultAzureCredential()

# Get AI project parameters from environment variables (matching evaluate.py)
project_endpoint = os.getenv("PROJECT_ENDPOINT")
deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")  
agent_id = os.getenv("AZURE_EXISTING_AGENT_ID")
agent_name = os.getenv("AZURE_EXISTING_AGENT_NAME")

print(f"Using project endpoint: {project_endpoint}")
print(f"Using deployment name: {deployment_name}")
print(f"Using agent ID: {agent_id}")
print(f"Using agent name: {agent_name}")

In [ ]:
# initialize the agents client
# This client will be used to interact with the Azure AI Agents service
project_client = AgentsClient(endpoint=project_endpoint, credential=credential)
print("Project client initialized.")

In [ ]:
agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a GenAI agent. Always answer in French",
)
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# Print agent details to verify correct targeting
print(f"We are going to start running Red Team evaluation against recently created agent:")
print(f"  - Agent ID: {agent.id}")
print(f"  - Agent Name: {agent.name}")
print(f"  - Using Model: {deployment_name}")

In [ ]:
def agent_callback(query: str) -> str:

    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")
    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content=query)
    # [END create_message]
    print(f"Created message, message ID: {message.id}")
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")


    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")
            return "Error: Agent run failed."
        messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
        for msg in messages:
            if msg.text_messages:
                    return msg.text_messages[0].text.value
        return "Could not get a response from the agent."

In [ ]:
# Instantiate your AI Red Teaming Agent
# Specifying risk categories and number of attack objectives per risk categories you want the AI Red Teaming Agent to cover
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ], 
    num_objectives=5, # optional, defaults to 10
)

In [ ]:
print("Starting Red Team scan...")
result = await red_team_agent.scan(
    target=agent_callback,
    scan_name="Agent-Scan",
    attack_strategies=[AttackStrategy.Flip],
)
print("Red Team scan complete.")